In [1]:
%load_ext autoreload
%autoreload 2

import findspark
import pyspark
import pyspark.sql.functions as sqlFunctions
# import matplotlib.pyplot as plt
import config
import summary
import etl

from operator import add
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler, StandardScaler
from dataset import CriteoDataSets, CriteoData

# %matplotlib inline

In [2]:
data = CriteoDataSets(config.SPARK_CONTEXT, config.SPARK_SQL_CONTEXT)
train = data.train if not config.DEBUG else data.debug
train_5m = data.train_5m if not config.DEBUG else data.debug

In [3]:
# Label data histogram
if config.DEBUG:
    summary.label_histogram(train)

([0, 1, 2], [77337, 22663])

In [4]:
def pretty_print_bin(bins, index):
    return "{:0,.3f} - {:0,.0f}".format(bins[i], bins[i + 1])


for col_num, histogram in summary.int_columns_histograms_iter(train):
    bins, counts = histogram
    print("Column %d:\n" % col_num)
    for i, count in enumerate(counts):
        print ("%20s: %s\n" % (pretty_print_bin(bins, i), "{:,}".format(count)))
    print("")

Column 2:

          0.000 - 56: 55,236

        55.600 - 111: 295

       111.200 - 167: 34

       166.800 - 222: 10

       222.400 - 278: 5

       278.000 - 334: 3

       333.600 - 389: 2

       389.200 - 445: 0

       444.800 - 500: 1

       500.400 - 556: 1


Column 3:

      -2.000 - 1,850: 98,530

   1,850.400 - 3,703: 1,397

   3,702.800 - 5,555: 52

   5,555.200 - 7,408: 12

   7,407.600 - 9,260: 7

  9,260.000 - 11,112: 0

 11,112.400 - 12,965: 1

 12,964.800 - 14,817: 0

 14,817.200 - 16,670: 0

 16,669.600 - 18,522: 1


Column 4:

       0.000 - 6,554: 80,863

  6,553.500 - 13,107: 11

 13,107.000 - 19,660: 1

 19,660.500 - 26,214: 20

 26,214.000 - 32,768: 0

 32,767.500 - 39,321: 0

 39,321.000 - 45,874: 0

 45,874.500 - 52,428: 0

 52,428.000 - 58,982: 1

 58,981.500 - 65,535: 2


Column 5:

          0.000 - 42: 79,210

         41.700 - 83: 1,178

        83.400 - 125: 44

       125.100 - 167: 8

       166.800 - 208: 12

       208.500 - 250: 6

       250.200 

In [5]:
categorical_counts = {col_name: counts for (col_name, counts) in summary.cat_column_counts_iter(train)}

In [6]:
if config.DEBUG:
    for col_name in train.categorical_column_names:
        counts = categorical_counts[col_name]
        counts.show()

+--------+-----+
|      C1|count|
+--------+-----+
|05db9164|50197|
|68fd1e64|16676|
|5a9ed9b0| 8329|
|8cf07265| 4887|
|be589b51| 3336|
|5bfa8ab5| 2364|
|87552397| 1806|
|f473b8dc| 1339|
|39af2607| 1151|
|ae82ea21|  873|
|9a89b36c|  767|
|17f69355|  611|
|241546e0|  563|
|09ca0b81|  472|
|75ac2fe6|  417|
|41edac3d|  357|
|439a44a4|  342|
|7e5c2ff4|  275|
|1464facd|  260|
|b455c6d7|  253|
+--------+-----+
only showing top 20 rows

+--------+-----+
|      C2|count|
+--------+-----+
|38a947a1|12895|
|09e68b86| 6826|
|80e26c9b| 4043|
|38d50e09| 3724|
|287130e0| 3269|
|4f25e98b| 3268|
|1cfdf714| 2903|
|207b2d81| 2366|
|08d6d899| 1988|
|d833535f| 1892|
|58e67aaf| 1757|
|942f9a8d| 1735|
|8947f767| 1701|
|8084ee93| 1408|
|0468d672| 1392|
|0a519c5c| 1272|
|39dfaa0d| 1142|
|2c16a946| 1138|
|e5fb1af3| 1126|
|78ccd99e| 1087|
+--------+-----+
only showing top 20 rows

+--------+-----+
|      C3|count|
+--------+-----+
|    null| 3935|
|d032c263| 3703|
|b00d1501| 1900|
|02cf9876| 1707|
|aa8c1539| 15

In [7]:
if config.DEBUG:
    column_distinct_counts = {col_name: summary.column_distinct_count(train, col_name) 
                              for col_name in train.df.columns}

In [8]:
# Distinct value count for categorical features
if config.DEBUG:
    for col_name in train.categorical_column_names:
        count = "{:,}".format(column_distinct_counts.get(col_name))
        print("Column %s: %s distinct values" % (col_name, count))

Column C1: 541 distinct values
Column C2: 497 distinct values
Column C3: 43,870 distinct values
Column C4: 25,184 distinct values
Column C5: 145 distinct values
Column C6: 12 distinct values
Column C7: 7,623 distinct values
Column C8: 257 distinct values
Column C9: 3 distinct values
Column C10: 10,997 distinct values
Column C11: 3,799 distinct values
Column C12: 41,312 distinct values
Column C13: 2,796 distinct values
Column C14: 26 distinct values
Column C15: 5,238 distinct values
Column C16: 34,617 distinct values
Column C17: 10 distinct values
Column C18: 2,548 distinct values
Column C19: 1,303 distinct values
Column C20: 4 distinct values
Column C21: 38,618 distinct values
Column C22: 11 distinct values
Column C23: 14 distinct values
Column C24: 12,335 distinct values
Column C25: 51 distinct values
Column C26: 9,527 distinct values


In [9]:
integer_column_stats = {name: stats for (name, stats) in summary.integer_column_stats_iter(train)}

In [10]:
def pretty_print_stats(stats):
    return "\n".join(["\t{}: {:0,.3f}".format(key, val) for (key, val) in stats.items()])

if config.DEBUG:
    for col_name in train.integer_column_names:
        stats = integer_column_stats[col_name]
        print("%s: %s\n" % (col_name, pretty_print_stats(stats)))

I1: 	skewness: 13.045
	kurtosis: 373.759
	stddev: 10.451
	mean: 3.769

I2: 	skewness: 7.264
	kurtosis: 100.708
	stddev: 401.523
	mean: 112.864

I3: 	skewness: 74.676
	kurtosis: 7,426.228
	stddev: 538.819
	mean: 40.745

I4: 	skewness: 5.891
	kurtosis: 115.569
	stddev: 10.836
	mean: 8.280

I5: 	skewness: 9.146
	kurtosis: 115.043
	stddev: 65,797.898
	mean: 17,592.599

I6: 	skewness: 11.409
	kurtosis: 251.298
	stddev: 371.776
	mean: 139.685

I7: 	skewness: 42.413
	kurtosis: 4,233.619
	stddev: 65.460
	mean: 15.222

I8: 	skewness: 68.854
	kurtosis: 5,577.523
	stddev: 46.542
	mean: 13.575

I9: 	skewness: 9.010
	kurtosis: 163.166
	stddev: 286.416
	mean: 125.295

I10: 	skewness: 1.125
	kurtosis: 2.495
	stddev: 0.677
	mean: 0.620

I11: 	skewness: 6.399
	kurtosis: 66.442
	stddev: 4.630
	mean: 2.400

I12: 	skewness: 42.705
	kurtosis: 3,362.307
	stddev: 5.328
	mean: 0.938

I13: 	skewness: 67.003
	kurtosis: 5,899.185
	stddev: 52.045
	mean: 11.608



In [11]:
row_count = summary.row_count(train)
print("Row count: %s" % "{:,}".format(row_count))

Row count: 100,000


In [12]:
# Pull just the mean values from the integer column stats
integer_column_means = {col_name: round(integer_column_stats[col_name]["mean"])
                        for col_name in train.integer_column_names}

In [13]:
cat_features = ["C5", "C9", "C22"]

In [14]:
scaled_df, scaler, crm = etl.transform_train(train, integer_column_means, cat_features, 
                                                  categorical_counts, row_count)

In [15]:
scaled_df.show()

+-----+--------------------+
|label|     features_scaled|
+-----+--------------------+
|    0|[0.12832207225118...|
|    0|[0.25664414450236...|
|    0|[0.25664414450236...|
|    0|[0.51328828900472...|
|    0|[0.38496621675354...|
|    0|[0.51328828900472...|
|    0|[0.51328828900472...|
|    1|[0.12832207225118...|
|    0|[0.51328828900472...|
|    0|[0.51328828900472...|
|    0|[0.51328828900472...|
|    0|[0.0,0.0149431176...|
|    1|[0.0,-0.002490519...|
|    1|[0.51328828900472...|
|    0|[0.0,0.1270165002...|
|    0|[0.51328828900472...|
|    1|[0.12832207225118...|
|    0|[0.0,0.0024905196...|
|    0|[0.0,0.0597724707...|
|    0|[0.89825450575827...|
+-----+--------------------+
only showing top 20 rows

